Importing All Required Libraries

In [1]:
import urllib.request
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # lbrary for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests # library to handle requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    certifi-2019.3.9           |           py36_0         149 KB  conda-forge
    conda-4.6.8                |           py36_0         876 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    openssl-1.1.1b             |       h14c3975_1         4.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0

The follow

Using URLLIB to get all information from the web page

In [2]:
wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(wiki)
soup = BeautifulSoup(page, 'html5lib')

Using BeautifulSoup to get table on the page

In [3]:
all_table = soup.find_all('table')
table = soup.find('table', class_='wikitable sortable')

In [4]:
POSTALS=[]
BOROUGHS=[]
NEIGHS=[]

for row in table.findAll("tr"): #para tudo que estiver em <tr>
    cells = row.findAll('td') #variável para encontrar <td>
    if len(cells)==3: #número de colunas
        POSTALS.append(cells[0].find(text=True)) #iterando sobre cada linha
        BOROUGHS.append(cells[1].find(text=True))
        NEIGHS.append(cells[2].find(text=True))

In [6]:
df = pd.DataFrame(index=POSTALS, columns=['Postal'])

df['Postal']=POSTALS
df['Borough']=BOROUGHS
df['Neighboorhood']=NEIGHS

df.head()

,Postal,Borough,Neighboorhood
M1A,M1A,Not assigned,Not assigned
M2A,M2A,Not assigned,Not assigned
M3A,M3A,North York,Parkwoods
M4A,M4A,North York,Victoria Village
M5A,M5A,Downtown Toronto,Harbourfront


In [7]:
df.replace('Not assigned', np.nan, inplace = True)
df.head()

,Postal,Borough,Neighboorhood
M1A,M1A,NaN,Not assigned
M2A,M2A,NaN,Not assigned
M3A,M3A,North York,Parkwoods
M4A,M4A,North York,Victoria Village
M5A,M5A,Downtown Toronto,Harbourfront


In [8]:
df.dropna(subset=["Borough"], axis=0, inplace = True)
df.reset_index(drop = True, inplace = True)
df.head()

,Postal,Borough,Neighboorhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [9]:
df2 = df.groupby(['Postal','Borough'])['Neighboorhood'].apply(', '.join).reset_index()
df2.head()

,Postal,Borough,Neighboorhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
path = 'http://cocl.us/Geospatial_data'
df3 = pd.read_csv(path)
df3.rename(columns={'Postal Code':'Postal'}, inplace=True)
df3.head()

,Postal,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df_all = pd.merge(df2, df3, on=['Postal'], how='inner')
df_all.head()

,Postal,Borough,Neighboorhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [12]:
address = 'Toronto City, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.7804679, -79.3857655250008.


Map of Toronto using latitude and longitude values

In [13]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_all['Latitude'], df_all['Longitude'], df_all['Borough'], df_all['Neighboorhood']):
    label = '{}, {}'.format(df_all, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork